In [1]:
import pandas as pd
import pickle
import numpy as np
import os
import time
import calendar

In [2]:
!pwd

/Users/xiaozhi/Downloads/Master_Thesis/old_contact_data


In [3]:
dir = [f for f in os.listdir("./") if f.endswith(".pkl")]
rating = pd.read_csv("./ratings25.csv")
users = rating["userId"].drop_duplicates()

In [4]:
def getNumberFromFileName(name):
    return name.split("_")[-1].split(".")[0]

def openContact(path):
    with open("./" + path, 'rb') as f:
        return pickle.load(f)

def storeContact(sc, f):
    contact_name = "userMovie/contact_" + str(getNumberFromFileName(f)) + ".pkl"
    with open("./" + contact_name, 'wb') as f:
        pickle.dump(sc, f)

In [5]:
def selectUserFromRating(users, contact):
    peers = list(contact.keys())
    selected_users = np.random.choice(users, len(peers), replace=False)
    user_peer_dict = {peers[i] : selected_users[i] for i in range(len(selected_users))}
    user_contact = {i:dict() for i in range(1, 28)}
    mm = set()
    for person in contact.keys():
        peer = user_peer_dict[person]
        for p, times in contact[person].items():
            if user_peer_dict.__contains__(p):
                for t in times:
                    n = int(t / (60 * 60)) + 1
                    mm.add(n)
                    if user_contact[n].__contains__(peer):
                        if user_contact[n][peer] is None:
                            user_contact[n][peer] = {user_peer_dict[p]}
                        else:
                            user_contact[n][peer].add(user_peer_dict[p])
                    else:
                        user_contact[n][peer] = {user_peer_dict[p]}
    return user_contact, selected_users


In [6]:
for f in dir:
    contact = openContact(f)
    user_contact, selected_users = selectUserFromRating(users, contact)
    r = rating[rating["userId"].isin(selected_users)]
    csv_name = "rating_" + str(len(selected_users)) +".csv"
    r.to_csv(csv_name, index=False)
    storeContact(user_contact, f)


In [7]:
r


,userId,movieId,rating,timestamp
910,4,1,3.0,1573944252
911,4,260,3.5,1573938415
912,4,296,4.0,1573938898
913,4,541,4.5,1573944303
914,4,589,4.0,1573939042
...,...,...,...,...
24998295,162529,1610,4.0,888183857
24998296,162529,1721,5.0,888180979
24998297,162529,1730,4.0,888181260
24998298,162529,2019,4.0,888182949
